In [1]:
import pandas as pd
import os
import numpy as np
import scipy as sp
import gensim
from smart_open import smart_open
import multiprocessing

## Import corpus and reinferred vectors

In [2]:
stopwordsFile = '/Users/eslt0101/Projects/EviDENce/ML/stopwords-nl-stopwords-iso_expanded_gensim_processed.txt'
corpus_dir = '/Users/eslt0101/Data/eScience/EviDENce/Data/NR-Teksts/EviDENce_NR_output_clean/TargetSize150'
corpus_token_file = 'GV_corpus_text_para_clean.txt'
corpus_text_file = 'GV_corpus_text_para_clean.txt'
corpus_filenames_file = 'GV_corpus_text_para_clean_filenames.txt'

full_ctokf = os.path.join(corpus_dir,corpus_token_file)
full_ctf = os.path.join(corpus_dir,corpus_text_file)
full_cff = os.path.join(corpus_dir,corpus_filenames_file) 

### import and instantiate corpus

In [3]:
with open(stopwordsFile) as f:
    dutch_stopwords = f.readlines()
    

In [4]:
dutch_stopwords = [w.rstrip() for  w in dutch_stopwords]

In [5]:
def read_corpus(corpus_token_file,labeled=False):
    with smart_open(corpus_token_file,'r') as tf:
        for i,text_line in enumerate(tf):
            if labeled :
                #yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(text_line), [fn_line.rstrip()])
                yield gensim.models.doc2vec.TaggedDocument([token for token in gensim.utils.simple_preprocess(text_line,deacc=True) if token not in dutch_stopwords], [i])
            else:
                yield [token for token in gensim.utils.simple_preprocess(text_line,deacc=True) if token not in dutch_stopwords]

In [6]:
def read_corpus_lookup(corpus_filenames_file, corpus_token_file) :
    with smart_open(corpus_filenames_file, 'r') as fnf, smart_open(corpus_token_file,'r') as tf:
        i=0
        for (fn_line,tf_line) in zip(fnf,tf):
            yield ([i],[fn_line.rstrip()],[tf_line])
            i+=1

In [7]:
corp = list(read_corpus(full_ctokf,labeled=True))

In [8]:
corp_lookup = list(read_corpus_lookup(full_cff,full_ctokf))

### load reinfered vectors from frozen model

In [10]:
reinf_corp_medvec= np.load('/Users/eslt0101/Projects/EviDENce/ML/reinferred_corpus_terms_para_150_medvec_clean.npy')



## Match corpus elements to reinferred vectors

In [11]:
corp[0]

TaggedDocument(words=['werk', 'ogenblik', 'staking', 'uitbrak', 'staakten', 'eisten', 'fabriek', 'leiding', 'verwalter', 'neeee', 'jood', 'joodse', 'man', 'christenman', 'fabriek', 'leidde', 'eiste', 'heette', 'vak', 'riem', 'afgooien', 'meestaakte', 'protesteerde', 'heette', 'riem', 'afgooide', 'riem', 'natuurlijk', 'afgooien', 'eisten', 'fabrikant', 'directeur', 'zeggen', 'waarnemer', 'eiste', 'ie', 'ie', 'fabriek', 'bleef', 'draaien', 'bleven', 'werken'], tags=[0])

In [13]:
reinf_corp_medvec[0]

array([ 2.57731199e-01,  1.70168594e-01, -1.25183798e-01, -8.66047516e-02,
        1.73720919e-01,  3.49351466e-01,  7.29339048e-02, -3.21497977e-01,
        3.66000712e-01,  2.58636042e-01, -1.14476145e-02,  1.03312854e-01,
       -3.07318404e-01,  5.80084734e-02,  1.01981144e-01,  1.03501927e-01,
        1.76830195e-01,  1.47357829e-01,  3.86988208e-01, -9.11408514e-02,
       -2.32280619e-01, -3.41521837e-02, -2.19330236e-01, -1.96764678e-01,
        3.41263324e-01,  6.24188423e-01,  1.91391036e-01,  1.01480220e-01,
        7.40191191e-02, -5.86134326e-02, -1.41129859e-01, -5.13205856e-01,
        3.64180654e-01,  1.42115772e-01,  8.89902283e-03, -2.19948769e-01,
        2.66363553e-03,  2.88990274e-01,  7.70323761e-02, -1.96533404e-01,
       -3.32536249e-04, -8.70668367e-02,  4.47694778e-01, -3.23115572e-01,
        6.06596302e-02,  2.19329484e-02,  1.17869447e-02,  7.07467109e-01,
        3.52850661e-03, -2.12826408e-01])

In [22]:
(corp_lookup[1][1][0])


'05_JKKV_2003_JACQUES_FURTH_conversation_clipped_150_paragraph_145-156_text.txt'

In [27]:
corp_longids = [ corp_lookup[i][1][0].rstrip().split(".")[0] for i in range(len(corp_lookup))]

In [28]:
corp_longids


['05_JKKV_2003_JACQUES_FURTH_conversation_clipped_150_paragraph_104-108_text',
 '05_JKKV_2003_JACQUES_FURTH_conversation_clipped_150_paragraph_145-156_text',
 '05_JKKV_2003_JACQUES_FURTH_conversation_clipped_150_paragraph_447-455_text',
 '05_JKKV_2003_JACQUES_FURTH_conversation_clipped_150_paragraph_474-479_text',
 '05_JKKV_2003_JACQUES_FURTH_conversation_clipped_150_paragraph_806-813_text',
 '05_JKKV_2003_JACQUES_FURTH_conversation_clipped_150_paragraph_78-85_text',
 '05_JKKV_2003_JACQUES_FURTH_conversation_clipped_150_paragraph_674-681_text',
 '05_JKKV_2003_JACQUES_FURTH_conversation_clipped_150_paragraph_606-616_text',
 '05_JKKV_2003_JACQUES_FURTH_conversation_clipped_150_paragraph_227-234_text',
 '05_JKKV_2003_JACQUES_FURTH_conversation_clipped_150_paragraph_386-396_text',
 '05_JKKV_2003_JACQUES_FURTH_conversation_clipped_150_paragraph_480-486_text',
 '05_JKKV_2003_JACQUES_FURTH_conversation_clipped_150_paragraph_642-651_text',
 '05_JKKV_2003_JACQUES_FURTH_conversation_clipped_150_

In [32]:
series_reinfVec = pd.Series(np.array([]))
for i in range(len(reinf_corp_medvec)):
    series_reinfVec[i] = reinf_corp_medvec[i]
    
    

In [34]:
vectors = {'fragment_id': corp_longids, 'reinferred_vector': series_reinfVec}

In [36]:
corpVectorList = pd.DataFrame(vectors,columns=['fragment_id','reinferred_vector'])


In [38]:
corpVectorList.to_pickle("corpReinfVectorList.pkl")

In [44]:
corpVectorList.to_csv("corpReinfVectorList.csv",index=False)

In [3]:
corpVectorList = pd.read_pickle("corpReinfVectorList.pkl")

In [5]:
corpVectorList[corpVectorList.fragment_id == 'GV_Overloon_Venray_04_conversation_clipped_150_paragraph_73-82_text']

,fragment_id,reinferred_vector
23319,GV_Overloon_Venray_04_conversation_clipped_150...,"[0.17496978491544724, 0.24661047011613846, -0...."


In [8]:
import numpy as np
import scipy.spatial as spatial

In [9]:
def cosdistance(v1,v2):
    d = np.array([1. - spatial.distance.cosine(v1,v2)])
    return d

In [10]:
def cosdistance_corp(vec,corpus_vectors):
    cosd_vec_corp =[]
    for i in range (len(corpus_vectors)):
        cd = cosdistance(vec,corpus_vectors[i])
        cosd_vec_corp.append(cd)
    cosd_vec_corp = np.array(cosd_vec_corp).flatten()
    return cosd_vec_corp

In [11]:
v = corpVectorList['reinferred_vector'][23319]

In [12]:
v

array([ 0.17496978,  0.24661047, -0.2182125 , -0.21446794,  0.38005535,
        0.11361311,  0.01279776, -0.35902126,  0.29261196, -0.35852639,
        0.23712046,  0.0199017 ,  0.17135593,  0.17474853,  0.33255412,
        0.30560888,  0.18714247, -0.06929281,  0.23114087, -0.22899316,
       -0.25356495, -0.10596573, -0.04105504,  0.01465664,  0.09464388,
        0.06895003, -0.25077038, -0.0560915 , -0.10522526,  0.14110931,
       -0.2265422 , -0.10600438,  0.35256982, -0.2015816 ,  0.0753191 ,
        0.11641441,  0.14924185, -0.05804925,  0.43423371,  0.24520379,
        0.00143331,  0.03730953,  0.13720734, -0.21132443, -0.15773798,
       -0.22179498,  0.04509449,  0.38822226, -0.1541082 , -0.0645334 ])

In [13]:
cdvcorp = cosdistance_corp(v,corpVectorList['reinferred_vector'])

In [14]:
cdvcorp

array([0.43447554, 0.39683714, 0.65916968, ..., 0.55134598, 0.54537176,
       0.33679429])

In [15]:
ranked_retrieval = np.argsort(cdvcorp)[::-1]

In [16]:
ret_cosd = cdvcorp[ranked_retrieval]

In [17]:
ret_cosd

array([ 1.        ,  0.80778295,  0.79162382, ..., -0.33139815,
       -0.3346749 , -0.43072231])

In [18]:
ranked_retrieval

array([23319, 11056,  2397, ..., 22399, 15933, 22276])

In [22]:
corpVectorList['fragment_id'][ranked_retrieval]

23319    GV_Overloon_Venray_04_conversation_clipped_150...
11056    GV_Overloon_evacuees_01_conversation_clipped_1...
2397     GV_Overloon_Venray_02_conversation_clipped_150...
16479    GV_Verhalis_kloosterzusters_06a_conversation_c...
3686     GV_Traktor_Russenoorlog_09_conversation_clippe...
18045    GV_GAR_bombardement_04_conversation_clipped_15...
24096    GV_Nijkerk_Vluchtelingen_02_conversation_clipp...
12678    GV_Overloon_evacuees_07_conversation_clipped_1...
25069    GV_Nijkerk_Vluchtelingen_07_conversation_clipp...
18350    GV_Kindermonument_Gaaspstraat_04_conversation_...
11493    GV_CaleidoscoopFilm_ingekwartierd_01_conversat...
19647    GV_Schiedam_gastgezin2_09_conversation_clipped...
20877    GV_Verhalis_kloosterzusters_07b_conversation_c...
17428    GV_Nijkerk_Vluchtelingen_09_conversation_clipp...
10301    GV_Overloon_evacuees_04_conversation_clipped_1...
19456    GV_Overloon_evacuees_09_conversation_clipped_1...
6593     GV_Overloon_Venray_06_conversation_clipped_150.

In [23]:
groundTruthEx = pd.read_pickle('/Users/eslt0101/Projects/REPOS/eScience/EviDENce/doc2vec/d2v_evaluation/groundTruthEx_pd.pkl')

In [24]:
groundTruthEx['Titel'][0]

'GV_DeJager_ReisvandeRazzia_interview_Welling_conversation_clipped_150_paragraph_248-253_text'

In [29]:
vecGT = groundTruthEx['reinfVec'][0]

In [28]:
corpVectorList[corpVectorList.fragment_id == 'GV_DeJager_ReisvandeRazzia_interview_Welling_conversation_clipped_150_paragraph_248-253_text']

,fragment_id,reinferred_vector
15450,GV_DeJager_ReisvandeRazzia_interview_Welling_c...,"[0.3392736315727234, 0.27179042994976044, -0.1..."


In [30]:
vec = corpVectorList.reinferred_vector[15450]

In [31]:
vec

array([ 0.33927363,  0.27179043, -0.16801026, -0.20380647,  0.13683586,
        0.08472264,  0.18424188, -0.18145876,  0.22445451, -0.00311114,
        0.01058206,  0.12031288, -0.26398021,  0.33625874,  0.03873563,
       -0.05989163,  0.05653091,  0.16971132,  0.15461932, -0.02199169,
       -0.36243436,  0.04165543,  0.04841177, -0.02837305,  0.09280476,
        0.23569371,  0.17932937,  0.08360341, -0.1932203 ,  0.04128065,
       -0.13684578, -0.11732881,  0.45511979, -0.1771264 ,  0.11421824,
        0.0995436 ,  0.05680378,  0.13380276,  0.07739257,  0.10761983,
        0.07975015,  0.04996086, -0.05538582, -0.02385679, -0.19928084,
       -0.37455796,  0.15882851,  0.51568201, -0.1577179 , -0.18011839])

In [32]:
vecGT

array([ 0.33927363,  0.27179043, -0.16801026, -0.20380647,  0.13683586,
        0.08472264,  0.18424188, -0.18145876,  0.22445451, -0.00311114,
        0.01058206,  0.12031288, -0.26398021,  0.33625874,  0.03873563,
       -0.05989163,  0.05653091,  0.16971132,  0.15461932, -0.02199169,
       -0.36243436,  0.04165543,  0.04841177, -0.02837305,  0.09280476,
        0.23569371,  0.17932937,  0.08360341, -0.1932203 ,  0.04128065,
       -0.13684578, -0.11732881,  0.45511979, -0.1771264 ,  0.11421824,
        0.0995436 ,  0.05680378,  0.13380276,  0.07739257,  0.10761983,
        0.07975015,  0.04996086, -0.05538582, -0.02385679, -0.19928084,
       -0.37455796,  0.15882851,  0.51568201, -0.1577179 , -0.18011839])

In [ ]:
   m    